#### Objective :
Create a Python script that fetches product data from the "Fake Store API" using the requests library. The goal is to retrieve and display information about a specific product based on its ID.
#### Setup:
Ensure the requests library is installed: pip install requests.
#### Instructions:
- Write a function fetch_product_data(product_id) that takes a product ID as an argument and returns the product data as a dictionary.
- Inside the function, use the requests.get() function to make a GET request to the API endpoint https://fakestoreapi.com/products/{product_id}.
- Check if the response status code is 200 (OK). If it is, use response.json() to parse the JSON response and return the product data dictionary. If not, print an error message.
#### Output:
- If the function successfully fetches the product data, print the following information:
  - Product Name
  - Product Category
  - Product Price
  - Product Description
- If the function fails to fetch the product data, print an error message.
#### Bonus:
- Modify the exercise to allow the user to input a product ID interactively. Fetch and display the data for the specified product ID.

### Part I : Fetch Data

In [ ]:
import requests as req
import psycopg2 as pg
from psycopg2 import sql
from datetime import datetime, timedelta
import random
from faker import Faker

def fetch_product_data(product_id):
    url = f"https://fakestoreapi.com/products/{product_id}"
    response = req.get(url)
    
    if response.status_code == 200:
        product_data = response.json()
        return product_data
    else:
        print(f"Failed to fetch product data. Status code: {response.status_code}")
        return None

def display_product_data(product_data):
    if product_data:
        print("Product Name:", product_data['title'])
        print("Product Category:", product_data['category'])
        print("Product Price: $", product_data['price'])
        print("Product Description:", product_data['description'])
    else:
        print("No product to display.")

if __name__ == "__main__":
    product_id = input("Enter the product ID: ")
    product_data = fetch_product_data(product_id)
    display_product_data(product_data)

#### Objective:
Practice working with APIs and generating random datetimes by creating a script that fetches product data from an API, generates random datetimes, and stores the information in a PostgreSQL database.
#### Instructions:
- Write a Python script that fetches product data from the "Fake Store API" using the requests library for a range of product IDs (e.g., from 1 to 10).
- For each product fetched, generate a random datetime within the past year and store the product data (title, category, price, description, and generated datetime) in a PostgreSQL database.
#### Implementation:
- Define a function fetch_product_data(product_id) that fetches product data for a given product ID from the API. Handle both successful and failed requests.
- Create a PostgreSQL database named 'products'.
- Define a function create_products_table(cursor) that creates a products table with columns for product information (title, category, price, description) and a date_added column for the generated datetimes.
- Define a function insert_product_into_db(cursor, product_data) that inserts a product's data along with a generated datetime into the products table.
- In the __main__ section, use the defined functions to fetch product data, generate random datetimes, and store the information in the database.
- Print a message indicating the success or failure of data retrieval and insertion.

### Part II : Creating a Product Database

In [ ]:
def create_database(new_db_name):
    conn = pg.connect(dbname='postgres', user='damien', password=1234, host='localhost', port=5432)
    conn.autocommit = True
    cursor = conn.cursor()
    cursor.execute(sql.SQL("CREATE DATABASE {}").format(sql.Identifier(new_db_name)))
    print(f"Database '{new_db_name}' created successfully.")
    
def create_connection(user, password, host, port, database):
    connection = pg.connect(user='damien', password='1234', host='localhost', port='5432', database='products')
    connection.autocommit = True
    cursor = connection.cursor()
    return connection, cursor

def generate_random_datetime():
    now = datetime.now()
    past_year = now - timedelta(days=365)
    return past_year + (now - past_year) * random.random()

def create_products_table(cursor):
    cursor.execute(""" CREATE TABLE IF NOT EXISTS products (
            id SERIAL PRIMARY KEY,
            title TEXT NOT NULL,
            category TEXT NOT NULL,
            price NUMERIC NOT NULL,
            description TEXT NOT NULL,
            date_added TIMESTAMP NOT NULL)""")

def insert_product_into_db(cursor, product_data, date_added):
    cursor.execute(""" INSERT INTO products (title, category, price, description, date_added)
        VALUES (%s, %s, %s, %s, %s)
    """, (product_data['title'], product_data['category'], product_data['price'], product_data['description'], date_added))

if __name__ == "__main__":
    try:
        create_database('products')
        
        connection, cursor = create_connection(user='damien', password='1234', host='localhost', port='5432', database='products')

        create_products_table(cursor)

        for product_id in range(1, 21):
            product_data = fetch_product_data(product_id)
            if product_data:
                date_added = generate_random_datetime()
                insert_product_into_db(cursor, product_data, date_added)
                print(f"Successfully inserted product ID {product_id} into database.")
            else:
                print(f"Skipping product ID {product_id} due to failed data retrieval.")

    except Exception as error:
        print("Error while connecting to PostgreSQL:", error)

    finally:
        if connection:
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed.")

#### Objective:
Practice working with databases, creating a command-line interface (CLI), and retrieving data to display to users by building a Warehouse CLI that connects to a product database.
#### Instructions:
- Write a Python script that establishes a connection to the products.db database.
- Implement a function display_items(cursor) that queries and displays all product items from the database.
- Create a main() function that runs a loop for user interaction. The loop should:
  - Display a menu with the following options:
    - Display Items
    - Exit
  - Prompt the user to select an option.
  - If the user selects "Display Items," call the display_items function to show the products.
  - If the user selects "Exit," break the loop and print "Goodbye!"

### Part III : Warehouse Command-Line Interface

In [ ]:
def display_items(cursor):
    cursor.execute("SELECT * FROM products")
    items = cursor.fetchall()
    if items:
        for item in items:
            print(f"ID: {item[0]}")
            print(f"Title: {item[1]}")
            print(f"Category: {item[2]}")
            print(f"Price: {item[3]}")
            print(f"Description: {item[4]}")
            print(f"Date Added: {item[5]}")
            print("-" * 40)
    else:
        print("No items found in the database.")

def main():
    try:
        connection, cursor = create_connection(user='damien', password='1234', host='localhost', port='5432', database='products')

        while True:
            print("\nMenu:")
            print("1. Display Items")
            print("2. Exit")
            choice = input("Select an option: ")

            if choice == '1':
                display_items(cursor)
            elif choice == '2':
                print("Goodbye!")
                break
            else:
                print("Invalid option, please try again.")

    except Exception as error:
        print("Error while connecting to PostgreSQL:", error)

    finally:
        if connection:
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed.")

if __name__ == "__main__":
    main()

#### Objective:
Practice working with databases, using the Faker library to generate random data, and populating a table with user information by creating a script that populates the users table in the products database.
#### Instructions:
- Write a Python script that establishes a connection to the products.db database.
- Implement a function create_users_table(cursor) that creates a users table with appropriate columns (id, user_name, first_name, last_name, address, password).
- Define a function generate_random_user_data(fake) that uses the Faker library to generate random user data (user_name, first_name, last_name, address, password).
- Define a function insert_user_into_db(cursor, user_data) that inserts a user's data into the users table.
- In the __main__ section, generate and insert random user data into the users table for a specified number of users.

### Part IV : Populate Users Table with Random Data

In [ ]:
def users_table(cursor):
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS users (
            id SERIAL PRIMARY KEY,
            user_name TEXT NOT NULL,
            first_name TEXT NOT NULL,
            last_name TEXT NOT NULL,
            address TEXT NOT NULL,
            password TEXT NOT NULL)""")

def user_data(fake):
    return {'user_name': fake.user_name(),
            'first_name': fake.first_name(),
            'last_name': fake.last_name(),
            'address': fake.address(),
            'password': fake.password()}

def db_user(cursor, user_data):
    cursor.execute("""
        INSERT INTO users (user_name, first_name, last_name, address, password)
        VALUES (%s, %s, %s, %s, %s)""",
        (user_data['user_name'], user_data['first_name'], user_data['last_name'], user_data['address'], user_data['password']))

if __name__ == "__main__":
    try:
        connection, cursor = create_connection(user='damien', password='1234', host='localhost', port='5432', database='products')

        users_table(cursor)
        connection.commit()

        fake = Faker()

        number_of_users = 15 
        for _ in range(number_of_users):
            users_data = user_data(fake)
            db_user(cursor, users_data)

        connection.commit()
        print(f"Successfully inserted {number_of_users} users into the database.")

    except Exception as error:
        print("Error while connecting to PostgreSQL:", error)

    finally:
        if connection:
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed.")

#### Objective:
Practice working with databases, user authentication, and command-line interfaces (CLIs) by creating a Warehouse CLI that requires user authentication using a PostgreSQL database.
#### Instructions:
- Write a Python script that establishes a connection to the products.db database.
- Implement a function display_items(cursor) that queries and displays all product items from the database.
- Define a function authenticate_user(cursor, user_name, password) that authenticates a user based on the provided username and password.
- Define a function login(cursor) that handles user authentication by taking user input for username and password and calling the authenticate_user function.
- Create a main() function that calls the login function and, if authentication is successful, allows the user to interact with the warehouse CLI.
- Inside the CLI loop, provide the options to display items and exit.

### Part V : Warehouse CLI with User Authentication

In [ ]:
def create_users_table(cursor):
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS users (
            id SERIAL PRIMARY KEY,
            user_name TEXT NOT NULL,
            first_name TEXT NOT NULL,
            last_name TEXT NOT NULL,
            address TEXT NOT NULL,
            password TEXT NOT NULL)""")

def display_items(cursor):
    cursor.execute("SELECT * FROM products")
    items = cursor.fetchall()
    if items:
        for item in items:
            print(f"ID: {item[0]}")
            print(f"Title: {item[1]}")
            print(f"Category: {item[2]}")
            print(f"Price: {item[3]}")
            print(f"Description: {item[4]}")
            print(f"Date Added: {item[5]}")
            print("-" * 40)
    else:
        print("No items found in the database.")

def authenticate_user(cursor, user_name, password):
    query = sql.SQL("SELECT * FROM users WHERE user_name = %s AND password = %s")
    cursor.execute(query, (user_name, password))
    user = cursor.fetchone()
    return user is not None

def login(cursor):
    user_name = input("Enter username: ")
    password = input("Enter password: ")
    if authenticate_user(cursor, user_name, password):
        print("Login successful!")
        return True
    else:
        print("Login failed. Please check your username and password.")
        return False

def main():
    try:
        connection, cursor = create_connection(user='damien', password='1234', host='localhost', port='5432', database='products')
        
        create_users_table(cursor)
        connection.commit()

        if login(cursor):
            while True:
                print("\nMenu:")
                print("1. Display Items")
                print("2. Exit")
                choice = input("Select an option: ")

                if choice == '1':
                    display_items(cursor)
                elif choice == '2':
                    print("Goodbye!")
                    break
                else:
                    print("Invalid option, please try again.")

    except Exception as error:
        print("Error while connecting to PostgreSQL:", error)

    finally:
        if connection:
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed.")

if __name__ == "__main__":
    main()